In [1]:
"""
Changes:
Mandatory:
- Multiclass classification
Optional:
- Updated word embedding model(s)
    - Word2Vec, ELMo
"""
"""
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-1-9f07a11e5f13>, line 14)

In [ ]:
vocabulary_size = 5000

(traindata, trainlabels), (testdata, testlabels) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(traindata), len(testdata)))

In [ ]:
print(traindata[6])
print("_________________")
print(trainlabels[6])

In [ ]:
word2id = imdb.get_word_index()
id2word = {i:word for word, i in word2id.items()}
print([id2word.get(i, ' ') for i in traindata[6]])
print("Label: {}".format(trainlabels[6]))

In [ ]:
max_words = 500
traindata = sequence.pad_sequences(traindata, maxlen=max_words)
testdata = sequence.pad_sequences(testdata, maxlen=max_words)

In [ ]:
embedding_size = 32
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
batch_size = 64
num_epochs = 3

valdata, vallabels = traindata[:batch_size], trainlabels[:batch_size]
traindata2, trainlabels2 = traindata[batch_size:], trainlabels[batch_size:]

model.fit(traindata2, trainlabels2, validation_data=(valdata,vallabels), batch_size=batch_size, epochs=num_epochs)


In [ ]:
scores = model.evaluate(testdata, testlabels, verbose=0)
print("Test accuracy:", scores[1])
"""

In [2]:
# Loading modules
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import random as rand
import gensim as gs

Using TensorFlow backend.


In [3]:
# Load data & preprocess
df = pd.read_csv("imdb_master.csv", encoding="latin-1")
#df.head(10)

df = df.sort_index(ascending=True, axis=0)

# Throw out unsupported or incorrectly labeled reviews
#for row in range(len(df)):
#    if df['label'][row] != "neg" and df['label'][row] != "pos":
df.drop(df.index[50000:], inplace=True)

#print(df['review'][48:55])
#print(df['label'][48:55])

# Attempt to simulate multi-class encoding by generating a random star rating for reviews based on pos/neg cls
for row in range(len(df)):
    if df['label'][row] == "neg":
        df['label'][row] = rand.randrange(1,4)
    elif df['label'][row] == "pos":
        df['label'][row] = rand.randrange(3,6)
    else:
        print("Category labelling error")
        
print(df['label'][32469])

max_input_words = 0        
# Find the longest review to pad the others to match its length
for row in range(len(df)):
    if len(df['review'][row].split()) > max_input_words:
        max_input_words = len(df['review'][row].split())
        
print(max_input_words)

if max_input_words == 0:
    max_input_words = 500

/home/brad/PycharmProjects/IMBDSentimentAnalysis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/brad/PycharmProjects/IMBDSentimentAnalysis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1
2470


In [4]:
# Generate word embeddings
w2v_model = gs.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [5]:
data = list()
for row in range(len(df)):
    review = []
    for w in df['review'][row].split():
        try:
            review.append(w2v_model.get_vector(w.lower()))
        except KeyError:
            continue
    data.append(review)
    #df['embedding'][row] = sentence
    #df['review'][row] = [w2v_model.get_vector(w.lower()) for w in df['review'][row].split()]
#data = np.array(df['embedding'])
data = np.array(data)
labels = np.array(df['label'])

print(len(data))
print(len(labels))
print(len(data[0][0]))

vocab_size=3000000
embedding_dim=300

"""
data = num. sentences (reviews)
data[0] = num. words in the sentence
data[0][0] = 300-dimensional word embedding
"""


50000
50000
300


'\ndata = num. sentences (reviews)\ndata[0] = num. words in the sentence\ndata[0][0] = 300-dimensional word embedding\n'

In [8]:
# Build the model
model = Sequential()
#model.add(Embedding(vocab_size, embedding_dim, input_length=max_input_words))
model.add(LSTM(100, input_dim=embedding_dim))#, input_length=max_input_words))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

/home/brad/PycharmProjects/IMBDSentimentAnalysis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
/home/brad/PycharmProjects/IMBDSentimentAnalysis/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, input_shape=(None, 300...)`
  after removing the cwd from sys.path.


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 160,501
Trainable params: 160,501
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# Cut out some of the data so it will fit into memory
np.random.shuffle(data)
data = data[:int(len(data)/32)]
labels = labels[:int(len(labels)/32)]
print(labels[0])
print(labels[int(len(labels)-1)])

# Train test split
traind, vald, trainl, vall = train_test_split(data, labels)

traind = sequence.pad_sequences(traind, maxlen=max_input_words)
vald = sequence.pad_sequences(vald, maxlen=max_input_words)

print(len(traind[0]))
print(len(vald[0]))

1
2
2470
2470


In [10]:
# Train the model

print(len(data))
print(len(labels))

model.fit(traind, trainl, verbose=1)

1562
1562
Epoch 1/1
1171/1171 [==============================] - 53s 45ms/step - loss: -6.2409 - accuracy: 0.3185

In [11]:
# Evaluate the model
model.evaluate(vald, vall, verbose=1)

391/391 [==============================] - 6s 17ms/step


[-13.664190982613722, 0.3324808180332184]

In [12]:
# Test the model by making predictions
print(model.predict(vald[:11]))


[[0.99999905]
 [0.99999905]
 [0.99999905]
 [0.99999905]
 [0.99999905]
 [0.99999905]
 [0.99999905]
 [0.99999905]
 [0.99999905]
 [0.99999905]
 [0.99999905]]
